# Webscraping allrecipes.com
<b>Edie Espejo</b>
<br>
2017-09-07

<b>Link</b>: http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201

# Install BeautifulSoup

In [2]:
# import pip
# pip.main(["install", "bs4"])

  Running setup.py bdist_wheel for bs4: started
  Running setup.py bdist_wheel for bs4: finished with status 'done'
  Stored in directory: C:\Users\ehze\AppData\Local\pip\Cache\wheels\84\67\d4\9e09d9d5adede2ee1c7b7e8775ba3fbb04d07c4f946f0e4f11
Successfully built bs4


0

In [12]:
# pip.main(["install", "requests_cache"])

0

In [17]:
# IMPORTANT LIBRARIES
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import requests_cache

# Collect Links
It seems that the links are formatted in this fashion:
http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5

where we can replace the last numbers to resemble integers from 2 to 9.

In [9]:
# FORMAT LINKS
base_link = "http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page="

all_links = [base_link+str(integer)+"#"+str(integer) for integer in range(2,10)]

In [10]:
all_links

['http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=2#2',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=3#3',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=4#4',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=6#6',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20re

The markup sort of looks like this.

        <h3 class="grid-col__h3 grid-col__h3--recipe-grid">
            Chef John's Banana Bread
                <div class="grid-col__video">
                    <a href="/video/528/chef-johns-banana-bread/" data-internal-referrer-link='hub recipe' data-click-id='cardslot 1'><span class="icon--videoplay-small-white"></span></a>
                </div>
        </h3>

In [57]:
# FUNCTION TO RETRIEVE URLS FROM ONE PAGE
def one_page_recipes(url):
    soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    page_recipes = soup.find_all("article", {"class", "grid-col--fixed-tiles"})
    
    # INITIALIZE LIST
    recipes = []
    for page in page_recipes:
        try:
            recipes.append(page.find_all("a")[0]["href"])
        except:
            recipes.append(0)
            
    # MISSING VALUES ARE RETURNED HERE        
    return(recipes)

In [58]:
fsu = one_page_recipes(all_links[0])

In [59]:
fsu

['/recipe/219967/chef-johns-banana-bread/',
 '/recipe/6754/lower-fat-banana-bread-i/',
 '/recipe/22722/brown-sugar-banana-nut-bread-i/',
 0,
 '/recipe/241707/joys-easy-banana-bread/',
 '/recipe/16706/banana-pumpkin-bread/',
 '/recipe/7116/banana-bread---quick-bread-for-machines/',
 '/recipe/6777/banana-nut-bread-i/',
 '/recipe/17050/hawaiian-banana-nut-bread/',
 '/recipe/16999/grandmas-homemade-banana-bread/',
 0,
 '/recipe/17013/banana-peanut-butter-bread/',
 '/recipe/24930/lower-fat-banana-bread-ii/',
 '/recipe/7163/favorite-banana-blueberry-quick-bread/',
 '/recipe/15746/banana-oatmeal-bread/',
 '/recipe/17656/rich-and-delicious-banana-bread/',
 '/recipe/16666/banana-bran-muffins/',
 '/recipe/141933/banana-zucchini-bread/',
 0,
 '/recipe/6894/banana-coconut-loaf/',
 '/recipe/24575/oatmeal-banana-nut-bread/',
 '/recipe/7141/banana-nut-bread-ii/',
 '/recipe/6690/mothers-banana-bread/']